## **Create `.osw` file**

Create a .osw file with the target peptide `AFVDFLSDEIK2`. This is done by converting the `.tsv` file with PXD017703 to a dummy `.osw` file. 

In [1]:
import pandas as pd
import numpy as np
import sqlite3
import shutil
import requests

#### **Fetch Data**

Data is fetched using the `fetchData.sh` script

In [2]:
# Copy the library to a new file, this will be what create the .osw file from
shutil.copy("190513_hela_24fr_library_ptypic_decoy.pqp", 'ionMobilityTest.osw')

'ionMobilityTest.osw'

In [3]:
conn = sqlite3.connect("ionMobilityTest.osw")

In [4]:
rslts = pd.read_csv("pyprophet_export_25pc_50ng.tsv", sep='\t')

In [5]:
rslts = rslts[(rslts['FullPeptideName'] == 'AFVDFLSDEIK') & (rslts['Charge'] == 2)].copy()

In [6]:
rslts

,transition_group_id,decoy,run_id,filename,IM,IM_MS1,RT,assay_rt,delta_rt,iRT,...,var_ms2_elution_model_fit_score,var_ms2_im_xcorr_shape,var_ms2_im_xcorr_coelution,var_ms2_im_delta_score,var_ms2_sonar_lag,var_ms2_sonar_shape,var_ms2_sonar_log_sn,var_ms2_sonar_log_diff,var_ms2_sonar_log_trend,var_ms2_sonar_rsq
19871,33018,0,5656776311985213027,20190816_TIMS05_MA_FlMe_diaPASEF_25pc_50ng_A2_...,0.979779,0.980456,6240.41,6240.400747,0.009253,66.989397,...,NaN,0.968553,1.396681,0.001254,NaN,NaN,NaN,NaN,NaN,NaN
19872,33018,0,5656776311985213027,20190816_TIMS05_MA_FlMe_diaPASEF_25pc_50ng_A2_...,0.979454,0.979969,6261.07,6240.405881,20.664119,68.841634,...,NaN,0.614877,9.156187,0.003512,NaN,NaN,NaN,NaN,NaN,NaN
19873,33018,0,5656776311985213027,20190816_TIMS05_MA_FlMe_diaPASEF_25pc_50ng_A2_...,0.975373,0.977215,6271.32,6240.405027,30.914973,69.760885,...,NaN,0.605793,7.177857,0.003481,NaN,NaN,NaN,NaN,NaN,NaN
19874,33018,0,2870707016753918864,20190816_TIMS05_MA_FlMe_diaPASEF_25pc_50ng_A2_...,0.978579,0.979768,6242.15,6242.145214,0.004786,63.637029,...,NaN,0.975977,1.155826,0.001074,NaN,NaN,NaN,NaN,NaN,NaN
19875,33018,0,2870707016753918864,20190816_TIMS05_MA_FlMe_diaPASEF_25pc_50ng_A2_...,0.981811,0.982960,6256.67,6242.142411,14.527589,63.872895,...,NaN,0.696450,4.527548,0.003774,NaN,NaN,NaN,NaN,NaN,NaN
19876,33018,0,295227301306857,20190816_TIMS05_MA_FlMe_diaPASEF_25pc_50ng_A2_...,0.980773,0.981425,6244.37,6244.379389,-0.009389,63.694808,...,NaN,0.974809,1.026186,0.003413,NaN,NaN,NaN,NaN,NaN,NaN
19877,33018,0,295227301306857,20190816_TIMS05_MA_FlMe_diaPASEF_25pc_50ng_A2_...,0.983123,0.986551,6258.42,6244.379603,14.040397,63.961188,...,NaN,0.687260,2.956088,0.005763,NaN,NaN,NaN,NaN,NaN,NaN


#### **Feature Table**

In [7]:
# create feature table
feature = rslts[['id', 'transition_group_id', 'run_id', 'assay_rt', 'delta_rt', 'assay_iRT', 'delta_iRT', 'leftWidth', 'rightWidth', 'IM']].copy()

In [8]:
feature['EXP_RT'] = feature['assay_rt'] + feature['delta_rt']
feature['NORM_RT'] = feature['assay_iRT'] + feature['delta_iRT']
feature = feature.rename(columns=dict(id='ID', run_id='RUN_ID', transition_group_id='PRECURSOR_ID', leftWidth='LEFT_WIDTH', rightWidth='RIGHT_WIDTH', IM='EXP_IM', delta_rt='DELTA_RT')).drop(columns=['assay_rt', 'assay_iRT', 'delta_iRT'])

In [9]:
feature.to_sql('FEATURE', conn, index=False)

7

---

#### **Feature MS2 - Exclude Scores**

In [10]:
feature_ms2 = rslts[['id', 'Intensity', 'aggr_Peak_Apex']].copy()
feature_ms2['aggr_Peak_Apex'] = feature_ms2['aggr_Peak_Apex'].apply(lambda x: max([ float(i) for i in x.split(';') ]))
feature_ms2 = feature_ms2.rename(columns = dict(id='FEATURE_ID', Intensity='AREA_INTENSITY', aggr_Peak_Apex='APEX_INTENSITY'))

In [11]:
feature_ms2.to_sql("FEATURE_MS2", conn, index=False)

7

#### **FEATUE MS1 - Exclude Scores**

In [12]:
feature_ms1 = rslts[['id', 'aggr_prec_Peak_Apex', 'aggr_prec_Peak_Area']].copy()
feature_ms1 = feature_ms1.rename(columns = dict(id='FEATURE_ID', aggr_prec_Peak_Area='AREA_INTENSITY', aggr_prec_Peak_Apex='APEX_INTENSITY'))

In [13]:
feature_ms1.to_sql("FEATURE_MS1", conn, index=False)

7

#### **Feature Transition**

In [14]:
feature_transition = rslts[['id', 'aggr_Peak_Area', 'aggr_Peak_Apex', 'aggr_Fragment_Annotation']].drop_duplicates().copy()

feature_transition = feature_transition.set_index(['id']).apply(lambda x: x.str.split(';').explode()).reset_index()
feature_transition['TRANSITION_ID'] = feature_transition['aggr_Fragment_Annotation'].str.extract(r'(\d*)').astype(int)

feature_transition = feature_transition.rename(
    dict(id='FEATURE_ID', aggr_Peak_Area='AREA_INTENSITY', aggr_Peak_Apex='APEX_INTENSITY'), axis=1).drop(columns='aggr_Fragment_Annotation')

In [15]:
feature_transition

,FEATURE_ID,AREA_INTENSITY,APEX_INTENSITY,TRANSITION_ID
0,-8695081159217853818,138434.0,50509.3248023987,228216
1,-8695081159217853818,755923.0,279267.348846436,228217
2,-8695081159217853818,642517.0,237738.337417603,228218
3,-8695081159217853818,606450.0,233409.041866302,228219
4,-8695081159217853818,1190100.0,477323.231594086,228220
5,-8695081159217853818,512767.0,201583.622556686,228221
6,3856844824901256445,1889.04,893.01416015625,228216
7,3856844824901256445,9482.97,2838.98555755615,228217
8,3856844824901256445,5687.99,1910.01319885254,228218
9,3856844824901256445,6293.98,1784.9877166748,228219


In [16]:
feature_transition.to_sql('FEATURE_TRANSITION', conn, index=False)

42

---

#### **Score MS2**

In [17]:
score_ms2 = rslts[['id', 'd_score', 'peak_group_rank', 'm_score']].drop_duplicates()

In [18]:
score_ms2 = score_ms2.rename(columns=dict(
    id='FEATURE_ID',
    d_score='SCORE',
    peak_group_rank='RANK',
    m_score='QVALUE'))
    

In [19]:
score_ms2.to_sql('SCORE_MS2', conn, index=False)

7

#### **Run**

In [20]:
run = rslts[['run_id', 'filename']].drop_duplicates()

In [21]:
# rename run to ion mobility test 
run.loc[run['filename'] == '20190816_TIMS05_MA_FlMe_diaPASEF_25pc_50ng_A2_1_26.d_mergeFiles.mzML', 'filename'] = 'ionMobilityTest2.mzML'

In [22]:
run = run.rename(columns=dict(run_id='ID', filename='FILENAME'))

In [23]:
run.to_sql('RUN', conn, index=False)

3

#### **Make Precursor Table Smaller**

In [24]:
precursor = pd.read_sql("select * from precursor", conn)

In [25]:
precursor = precursor[precursor['ID'] == rslts['transition_group_id'].iloc[0]]

In [26]:
precursor.to_sql('PRECURSOR', conn, index=False, if_exists='replace')

1

---

#### **Make Transition Table Smaller**

In [27]:
transition = pd.read_sql('select * from transition', conn)

In [28]:
transition = transition[transition['ID'].isin(feature_transition['TRANSITION_ID'].drop_duplicates().values)]

In [29]:
transition.to_sql('TRANSITION', conn, index=False, if_exists='replace')

6

---

#### **Make Peptide Table Smaller**

In [30]:
peptide = pd.read_sql('select * from peptide', conn)

In [31]:
peptide = peptide[peptide['MODIFIED_SEQUENCE'] == rslts['FullPeptideName'].iloc[0]]

In [32]:
peptide.to_sql('PEPTIDE', conn, index=False, if_exists='replace')

1

---

#### **Make Protein Table Smaller**

In [33]:
protein = pd.read_sql('select * from Protein', conn)

In [34]:
protein = protein[protein['PROTEIN_ACCESSION'] == rslts['ProteinName'].iloc[0]]

In [35]:
protein

,ID,PROTEIN_ACCESSION,DECOY
12219,12219,sp|Q07021|C1QBP_HUMAN,0


In [36]:
protein.to_sql('PROTEIN', conn, index=False, if_exists='replace')

1

---

#### **Make Transition Precursor Mapping Table Smaller**

In [37]:
transition_precursor_mapping = pd.read_sql('select * from transition_precursor_mapping', conn)

In [38]:
transition_precursor_mapping = transition_precursor_mapping[transition_precursor_mapping['TRANSITION_ID'].isin(feature_transition['TRANSITION_ID'].drop_duplicates().values)]

In [39]:
transition_precursor_mapping

,TRANSITION_ID,PRECURSOR_ID
228216,228216,33018
228217,228217,33018
228218,228218,33018
228219,228219,33018
228220,228220,33018
228221,228221,33018


In [40]:
transition_precursor_mapping.to_sql('TRANSITION_PRECURSOR_MAPPING', conn, index=False, if_exists='replace')

6

---

#### **Make Precursor Peptide Mapping Smaller**

In [41]:
precursor_peptide_mapping = pd.read_sql("select * from precursor_peptide_mapping", conn)

In [42]:
precursor_peptide_mapping = precursor_peptide_mapping[precursor_peptide_mapping['PRECURSOR_ID'] == rslts['transition_group_id'].iloc[0]] 

In [43]:
precursor_peptide_mapping

,PRECURSOR_ID,PEPTIDE_ID
38036,33018,6558


In [44]:
precursor_peptide_mapping.to_sql('PRECURSOR_PEPTIDE_MAPPING', conn, index=False, if_exists='replace')

1

---

#### **Add Score Peptide**

In [45]:
score_peptide = rslts[['run_id', 'm_score_peptide_global']]
score_peptide = score_peptide.rename(columns={'run_id':'RUN_ID', 'm_score_peptide_global':'QVALUE'})
score_peptide['CONTEXT'] = 'global'
score_peptide['PEPTIDE_ID'] = 6558

In [46]:
score_peptide.to_sql("SCORE_PEPTIDE", conn, index=False, if_exists='replace')

7

#### **Add Score Protein**

In [47]:
score_protein = rslts[['run_id', 'm_score_protein_global']]
score_protein = score_protein.rename(columns={'run_id':'RUN_ID', 'm_score_protein_global':'QVALUE'})
score_protein['PROTEIN_ID'] = 12219
score_protein['CONTEXT'] = 'global'

In [48]:
score_protein.to_sql("SCORE_PROTEIN", conn, index=False, if_exists='replace')

7

#### **Clean Up Free Memory**

In [49]:
conn.execute('vacuum')

In [50]:
conn.close()